## 新电表记录数据

In [ ]:
import numpy as np
import pandas as pd

new_meter = pd.read_csv('../data/扩容电表.csv')
new_meter = new_meter[['记录日期', '反向有功总电能(kWh)', '正向有功总电能(kWh)']]
new_meter.rename(columns={'记录日期': 'ts', '反向有功总电能(kWh)': 'chg', '正向有功总电能(kWh)': 'dhg'}, inplace=True)
new_meter['ts'] = pd.to_datetime(new_meter['ts'])  # 转换为时间格式
new_meter = new_meter.sort_values(by='ts', ascending=True)  # 按时间升序排列
new_meter = new_meter.set_index('ts')  # 设置时间为索引
new_meter

## 按天计算谷充与峰放

In [ ]:
# 构建按天分组的数据
new_meter_daily = pd.DataFrame()
# 获取日期列表
date_list = sorted(set(new_meter.index.date))
new_meter_daily['ts'] = date_list
# chg列，当天0点-8点，谷时充电量
chg_list = []
for date in date_list:
    start = new_meter.loc[f"{date} 00:00:00":f"{date} 00:00:30"]
    end = new_meter.loc[f"{date} 08:00:00":f"{date} 08:00:30"]
    if start.empty or end.empty:
        chg_list.append(0)
        continue
    else:
        chg_list.append(end['chg'].values[0] - start['chg'].values[0])
new_meter_daily['chg'] = chg_list
# dhg列，当天17点-22点，峰时放电量
dhg_list = []
for date in date_list:
    start = new_meter.loc[f"{date} 17:00:00":f"{date} 17:00:30"]
    end = new_meter.loc[f"{date} 22:00:00":f"{date} 22:00:30"]
    if start.empty or end.empty:
        dhg_list.append(0)
        continue
    else:
        dhg_list.append(end['dhg'].values[0] - start['dhg'].values[0])
new_meter_daily['dhg'] = dhg_list
# 处理数据
new_meter_daily

## 扩容电池的衰减

In [ ]:
import plotly.graph_objs as go

# 添加等效容量列
new_meter_daily['capacity'] = np.sqrt(new_meter_daily['chg'] * new_meter_daily['dhg'])
# 获取初始与当前容量
initial = new_meter_daily.head(10).sort_values(by="capacity", ascending=False).iloc[0]
current = new_meter_daily.tail(10).sort_values(by="capacity", ascending=False).iloc[0]
# 计算衰减
attenuation = round((1 - current["capacity"] / initial["capacity"]) * 100, 2)
# 绘图
fig = go.Figure()
fig.update_layout(title=f"扩容电池衰减{attenuation}%")
fig.add_trace(go.Scatter(x=new_meter_daily['ts'], y=new_meter_daily['capacity'], mode='markers'))
fig.add_trace(
    go.Scatter(x=[initial['ts'], current['ts']], y=[initial['capacity'], current['capacity']], mode='markers+lines'))
fig.show()

## 初始化数据库

In [1]:
from sqlalchemy import create_engine

# 初始化数据库连接
host = "3.3.9.113"
port = "3306"
database = "EbPackTemp"
user = "root"
password = "0"
engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}:{port}/{database}")

## 每日充放电量修正

In [ ]:
# from sqlalchemy import text
# 
# connection = engine.connect()
# with connection.begin() as trans:
#     # 遍历更新
#     for row in new_meter_daily.iterrows():
#         ts = row[1]['ts']
#         chg = row[1]['chg']
#         dhg = row[1]['dhg']
#         # 执行更新
#         sql = f"update tc1_electric_quantity_day set valleye_sum_chg=valleye_sum_chg-{chg}, peake2_sum_dhg=peake2_sum_dhg-{dhg} where datetime='{ts}';"
#         connection.execute(text(sql))
#         print(f"更新{ts}数据成功")
#     # 提交事务
#     trans.commit()
# connection.close()